In [1]:
# Import the dependencies.
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
# Create a set of random latitude and longitude combinations.
lats = np.random.uniform(low=-90.000, high=90.000, size=1500)
lngs = np.random.uniform(low=-180.000, high=180.000, size=1500)
lat_lngs = zip(lats, lngs)
lat_lngs


In [3]:
# Add the latitudes and longitudes to a list.
coordinates = list(lat_lngs)

In [4]:
from citipy import citipy

In [5]:
# Create a list for holding the cities.
cities = []
# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count.
len(cities)

616

In [6]:
# Import dependencies 
import requests
from config import weather_api_key
from datetime import datetime

In [10]:
# Create an empty list to hold the weather data.
city_data = []
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1

# Loop through all the cities in our list.
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
    # Create endpoint URL with each city.
    url = "http://api.openweathermap.org/data/2.5/weather?units=imperial&APPID=" + weather_api_key
    city_url = url + "&q=" + city.replace(" ","+")

    # Log thr url, record, and set numbers and city
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record
    record_count+= 1
    # Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          "Date": city_date})

# If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")



Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | strezhevoy
Processing Record 2 of Set 1 | jamestown
Processing Record 3 of Set 1 | huanren
Processing Record 4 of Set 1 | ponta do sol
Processing Record 5 of Set 1 | salinas
Processing Record 6 of Set 1 | vanimo
Processing Record 7 of Set 1 | ushuaia
Processing Record 8 of Set 1 | albany
Processing Record 9 of Set 1 | hofn
Processing Record 10 of Set 1 | liaoyang
Processing Record 11 of Set 1 | sisimiut
Processing Record 12 of Set 1 | mergui
City not found. Skipping...
Processing Record 13 of Set 1 | yulara
Processing Record 14 of Set 1 | luderitz
Processing Record 15 of Set 1 | codrington
Processing Record 16 of Set 1 | mahebourg
Processing Record 17 of Set 1 | nikolskoye
Processing Record 18 of Set 1 | rikitea
Processing Record 19 of Set 1 | luganville
Processing Record 20 of Set 1 | haines junction
Processing Record 21 of Set 1 | cape town
Processing Record 22 of Set 1 | taolanaro
City not fou

In [11]:
city_data_df = pd.DataFrame(city_data)
city_data_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Strezhevoy,60.7333,77.5889,33.78,98,100,5.28,RU,2021-09-21 20:26:40
1,Jamestown,42.0970,-79.2353,74.01,70,75,4.00,US,2021-09-21 20:26:41
2,Huanren,41.2647,125.3667,56.37,97,100,4.09,CN,2021-09-21 20:26:41
3,Ponta Do Sol,32.6667,-17.1000,74.03,72,72,5.19,PT,2021-09-21 20:26:41
4,Salinas,36.6777,-121.6555,102.56,32,1,10.36,US,2021-09-21 20:22:43


In [14]:
new_column_order = ["City", "Country", "Date", "Lat", "Lng", "Max Temp", "Humidity", "Cloudiness", "Wind Speed"]
city_data_df = city_data_df[new_column_order]
city_data_df.head()

,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,Strezhevoy,RU,2021-09-21 20:26:40,60.7333,77.5889,33.78,98,100,5.28
1,Jamestown,US,2021-09-21 20:26:41,42.0970,-79.2353,74.01,70,75,4.00
2,Huanren,CN,2021-09-21 20:26:41,41.2647,125.3667,56.37,97,100,4.09
3,Ponta Do Sol,PT,2021-09-21 20:26:41,32.6667,-17.1000,74.03,72,72,5.19
4,Salinas,US,2021-09-21 20:22:43,36.6777,-121.6555,102.56,32,1,10.36


In [15]:
# Create the output file (CSV).
output_data_file = "weather_data/cities.csv"
# Export the City_Data into a CSV.
city_data_df.to_csv(output_data_file, index_label="City_ID")